In [32]:
import mediapipe as mp
import cv2
import matplotlib.pyplot as plt


In [37]:
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np

MARGIN = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54) # vibrant green

In [38]:
# STEP 3: Load the input image.
image = mp.Image.create_from_file("images/image1.jpeg")

In [39]:
# STEP 1: Import the necessary modules.
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import pandas as pd

# STEP 2: Create an HandLandmarker object.
base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
options = vision.HandLandmarkerOptions(base_options=base_options,
                                       num_hands=2)
detector = vision.HandLandmarker.create_from_options(options)

In [40]:
# STEP 4: Detect hand landmarks from the input image.
detection_result = detector.detect(image)

/home/joao/anaconda3/lib/python3.9/site-packages/google/protobuf/symbol_database.py:78: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [44]:
hand_landmarks_list = detection_result.hand_landmarks
handedness_list = detection_result.handedness

In [46]:
print(hand_landmarks_list[0])

[NormalizedLandmark(x=0.48669159412384033, y=0.841986894607544, z=3.379066697561939e-07, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.39080435037612915, y=0.743610143661499, z=-0.05455343425273895, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.31221964955329895, y=0.6146195530891418, z=-0.08601563423871994, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.24279066920280457, y=0.5166422128677368, z=-0.11597941815853119, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.165911465883255, y=0.46417614817619324, z=-0.1460321843624115, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.4450366497039795, y=0.4514409899711609, z=-0.038028448820114136, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.43562132120132446, y=0.3063657283782959, z=-0.07086575776338577, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.4349901080131531, y=0.21850693225860596, z=-0.10189125686883926, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.4374980330467224, y=0.14108